# Day 13: Mine Cart Madness
[link](https://adventofcode.com/2018/day/13)

## Parsing inputs

In [1]:
test_in = """/->-\\        
|   |  /----\\
| /-+--+-\\  |
| | |  | v  |
\-+-/  \\-+--/
  \------/   """.split('\n')

print('\n'.join(test_in))
print(set(len(s) for s in test_in))

/->-\        
|   |  /----\
| /-+--+-\  |
| | |  | v  |
\-+-/  \-+--/
  \------/   
{13}


In [2]:
with open('input.txt') as file:
  puzzle_in =[line.strip('\n') for line in file]

print(set(len(s) for s in puzzle_in))

{150}


## Part 1

Find `X,Y` coordinates of the first crash.

In [3]:
# Direction after a curve
curve_turns = {'>/':'^','>\\':'v','</':'v','<\\':'^','v/':'<','v\\':'>','^/':'>','^\\':'<'}

# Direction after an intersection depending on whether the cart wants to go left 'l' or right 'r'
intersection_turns = {'>l':'^','>r':'v','<l':'v','<r':'^','vr':'<','vl':'>','^r':'>','^l':'<'}

# Position increment depending on the cart's direction
position_increment = {'>': (1,0),'<': (-1,0),'^': (0,-1),'v': (0,1)}


# Generator of turn directions, which infinitely produces 'l', 's', 'r', 's', ...
def turn_direction_generator():
  while True:
    for turn in 'lsrs':
      yield turn


class Cart:
  def __init__(self, direction):
    self.direction = direction
    self.turn_direction_generator = turn_direction_generator()


  def update_direction(self, track):
    if track == '+':  # Intersection
      turn_direction = next(self.turn_direction_generator)
      if turn_direction in 'lr':  # Turn either left or right
        self.direction = intersection_turns[self.direction + turn_direction]
      else:  # Go straight - no need to change direction
        assert turn_direction == 's'
    elif track in '\/':  # Curve
      self.direction = curve_turns[self.direction + track]
    else:  # Straight
      assert track in '-|'


  def next_position(self, x, y):
    increment = position_increment[self.direction]
    return x+increment[0], y+increment[1]


  def __repr__(self):
    return self.direction

In [4]:
class Mine:
  def __init__(self, input_str):
    self.carts = {}
    self.tracks = []

    for y,line in enumerate(input_str):
      track_line = []
      for x,char in enumerate(line):
        if char in '<v>^':
          self.carts[(y,x)] = Cart(char)
          track_line.append('|' if char in '^v' else '-')
        else:
          track_line.append(char)
      self.tracks.append(track_line)


  def __repr__(self):
    return '\n'.join(''.join(char if (y,x) not in self.carts else self.carts[(y,x)].direction for x, char in enumerate(line)) for y,line in enumerate(self.tracks))


  def tick(self):
    new_carts = {}
    for (y,x) in sorted(self.carts.keys()):
      cart = self.carts[(y,x)]
      new_x,new_y = cart.next_position(x,y)

      track = self.tracks[new_y][new_x]
      cart.update_direction(track)

      if (new_y,new_x) in new_carts:  # Collision
        new_carts[(new_y,new_x)].direction = 'X'
        self.carts = new_carts
        raise Exception(f'Collision at ({new_x},{new_y})')

      new_carts[(new_y,new_x)] = cart

    self.carts = new_carts

In [5]:
test_mine = Mine(test_in)
assert str(test_mine)=='\n'.join(test_in)
print(f'00 {test_mine.carts}\n{test_mine}\n')

for i in range(20):
  test_mine.tick()
  print(f'{i+1:02} {test_mine.carts}\n{test_mine}\n')

00 {(0, 2): >, (3, 9): v}
/->-\        
|   |  /----\
| /-+--+-\  |
| | |  | v  |
\-+-/  \-+--/
  \------/   

01 {(0, 3): >, (4, 9): >}
/-->\        
|   |  /----\
| /-+--+-\  |
| | |  | |  |
\-+-/  \->--/
  \------/   

02 {(0, 4): v, (4, 10): >}
/---v        
|   |  /----\
| /-+--+-\  |
| | |  | |  |
\-+-/  \-+>-/
  \------/   

03 {(1, 4): v, (4, 11): >}
/---\        
|   v  /----\
| /-+--+-\  |
| | |  | |  |
\-+-/  \-+->/
  \------/   

04 {(2, 4): >, (4, 12): ^}
/---\        
|   |  /----\
| /->--+-\  |
| | |  | |  |
\-+-/  \-+--^
  \------/   

05 {(2, 5): >, (3, 12): ^}
/---\        
|   |  /----\
| /-+>-+-\  |
| | |  | |  ^
\-+-/  \-+--/
  \------/   

06 {(2, 6): >, (2, 12): ^}
/---\        
|   |  /----\
| /-+->+-\  ^
| | |  | |  |
\-+-/  \-+--/
  \------/   

07 {(2, 7): >, (1, 12): <}
/---\        
|   |  /----<
| /-+-->-\  |
| | |  | |  |
\-+-/  \-+--/
  \------/   

08 {(1, 11): <, (2, 8): >}
/---\        
|   |  /---<\
| /-+--+>\  |
| | |  | |  |
\-+-/  \-+--/
  \------

Exception: Collision at (7,3)

In [8]:
puzzle_mine = Mine(puzzle_in)
assert str(puzzle_mine)=='\n'.join(puzzle_in)  # Check if the initial tracks + carts representation is the same as the puzzle input

for i in range(2000):
  puzzle_mine.tick()

Exception: Collision at (36,87)

**Incorrect answers:**
- `35,50` is not the right answer
- `36,87` is not the right answer

In [6]:
test_in_2 = """
""".split('\n')

print('\n'.join(test_in_2))
print(set(len(s) for s in test_in_2))

/->-\        
|   |  /----\
| /-+--+-\  |
| | |  | |  |
\-+-/  \-+--/
  \------/   
{13}


In [7]:
test_mine_2 = Mine(test_in_2)
assert str(test_mine_2)=='\n'.join(test_in_2)
print(f'00 {test_mine_2.carts}\n{test_mine_2}\n')

for i in range(50):
  test_mine_2.tick()
  print(f'{i+1:02} {test_mine_2.carts}\n{test_mine_2}\n')

00 {(0, 2): >}
/->-\        
|   |  /----\
| /-+--+-\  |
| | |  | |  |
\-+-/  \-+--/
  \------/   

01 {(0, 3): >}
/-->\        
|   |  /----\
| /-+--+-\  |
| | |  | |  |
\-+-/  \-+--/
  \------/   

02 {(0, 4): v}
/---v        
|   |  /----\
| /-+--+-\  |
| | |  | |  |
\-+-/  \-+--/
  \------/   

03 {(1, 4): v}
/---\        
|   v  /----\
| /-+--+-\  |
| | |  | |  |
\-+-/  \-+--/
  \------/   

04 {(2, 4): >}
/---\        
|   |  /----\
| /->--+-\  |
| | |  | |  |
\-+-/  \-+--/
  \------/   

05 {(2, 5): >}
/---\        
|   |  /----\
| /-+>-+-\  |
| | |  | |  |
\-+-/  \-+--/
  \------/   

06 {(2, 6): >}
/---\        
|   |  /----\
| /-+->+-\  |
| | |  | |  |
\-+-/  \-+--/
  \------/   

07 {(2, 7): >}
/---\        
|   |  /----\
| /-+-->-\  |
| | |  | |  |
\-+-/  \-+--/
  \------/   

08 {(2, 8): >}
/---\        
|   |  /----\
| /-+--+>\  |
| | |  | |  |
\-+-/  \-+--/
  \------/   

09 {(2, 9): v}
/---\        
|   |  /----\
| /-+--+-v  |
| | |  | |  |
\-+-/  \-+--/
  \------/   



In [13]:
with open('test_2.txt') as file:
  test_3_in =[line.strip('\n') for line in file]

print(set(len(s) for s in test_3_in))
print('\n'.join(test_3_in))

{4}
/>\ 
|/+\
|\+/
\-/ 


In [14]:
test_mine_3 = Mine(test_3_in)
assert str(test_mine_3)=='\n'.join(test_3_in)
print(f'00 {test_mine_3.carts}\n{test_mine_3}\n')

for i in range(100):
  test_mine_3.tick()
  print(f'{i+1:02} {test_mine_3.carts}\n{test_mine_3}\n')

00 {(0, 1): >}
/>\ 
|/+\
|\+/
\-/ 

01 {(0, 2): v}
/-v 
|/+\
|\+/
\-/ 

02 {(1, 2): >}
/-\ 
|/>\
|\+/
\-/ 

03 {(1, 3): v}
/-\ 
|/+v
|\+/
\-/ 

04 {(2, 3): <}
/-\ 
|/+\
|\+<
\-/ 

05 {(2, 2): <}
/-\ 
|/+\
|\</
\-/ 

06 {(2, 1): ^}
/-\ 
|/+\
|^+/
\-/ 

07 {(1, 1): >}
/-\ 
|>+\
|\+/
\-/ 

08 {(1, 2): v}
/-\ 
|/v\
|\+/
\-/ 

09 {(2, 2): v}
/-\ 
|/+\
|\v/
\-/ 

10 {(3, 2): <}
/-\ 
|/+\
|\+/
\-< 

11 {(3, 1): <}
/-\ 
|/+\
|\+/
\</ 

12 {(3, 0): ^}
/-\ 
|/+\
|\+/
^-/ 

13 {(2, 0): ^}
/-\ 
|/+\
^\+/
\-/ 

14 {(1, 0): ^}
/-\ 
^/+\
|\+/
\-/ 

15 {(0, 0): >}
>-\ 
|/+\
|\+/
\-/ 

16 {(0, 1): >}
/>\ 
|/+\
|\+/
\-/ 

17 {(0, 2): v}
/-v 
|/+\
|\+/
\-/ 

18 {(1, 2): >}
/-\ 
|/>\
|\+/
\-/ 

19 {(1, 3): v}
/-\ 
|/+v
|\+/
\-/ 

20 {(2, 3): <}
/-\ 
|/+\
|\+<
\-/ 

21 {(2, 2): <}
/-\ 
|/+\
|\</
\-/ 

22 {(2, 1): ^}
/-\ 
|/+\
|^+/
\-/ 

23 {(1, 1): >}
/-\ 
|>+\
|\+/
\-/ 

24 {(1, 2): v}
/-\ 
|/v\
|\+/
\-/ 

25 {(2, 2): v}
/-\ 
|/+\
|\v/
\-/ 

26 {(3, 2): <}
/-\ 
|/+\
|\+/
\-< 

27 {(3, 1): <}
/-\ 
|/+\
|\+